# Water fluoridation and Dental Caries

**Date:** 2021-12-01

**Reference:** M249, Book 1, Part 2

In [1]:
suppressPackageStartupMessages(library(tidyverse))
library(R249)
library(DescTools)

## Summary

## Get the data

In [2]:
(dat <- as_tibble(read.csv(file = "..\\..\\data\\dentalcaries.csv")))

count,exposure,outcome,level
<int>,<chr>,<chr>,<chr>
5,fluoridated,caries,age 8 years
25,fluoridated,no caries,age 8 years
8,not fluoridated,caries,age 8 years
23,not fluoridated,no caries,age 8 years
0,fluoridated,caries,age 9 years
17,fluoridated,no caries,age 9 years
17,not fluoridated,caries,age 9 years
33,not fluoridated,no caries,age 9 years
5,fluoridated,caries,age 10 years


## Prepare the data

Cast the `exposure`, `outcome`, `level` columns to factors.

In [4]:
labexp <- c("not fluoridated", "fluoridated")
labout <- c("caries", "no caries")
lablev <- c("age 8 years", "age 9 years", "age 10 years", "age 11-12 years")
(sorteddat <- dat %>%
    mutate(exposure = factor(dat$exposure, labexp)) %>%
    mutate(outcome = factor(dat$outcome, labout)) %>%
    mutate(level = factor(dat$level, lablev)) %>%
    arrange(level, exposure, outcome))

count,exposure,outcome,level
<int>,<fct>,<fct>,<fct>
8,not fluoridated,caries,age 8 years
23,not fluoridated,no caries,age 8 years
5,fluoridated,caries,age 8 years
25,fluoridated,no caries,age 8 years
17,not fluoridated,caries,age 9 years
33,not fluoridated,no caries,age 9 years
0,fluoridated,caries,age 9 years
17,fluoridated,no caries,age 9 years
24,not fluoridated,caries,age 10 years


Filter the tibble on each specific `level`, pull the `count` column as a vector and initilise a matrix.
Append this new matrix to an array.

In [5]:
age8 <- filter(sorteddat, level == "age 8 years") %>%
    pull(count) %>%
    matrix(nrow = 2, ncol = 2, byrow = TRUE, dimnames = list(labexp, labout))
age9 <- filter(sorteddat, level == "age 9 years") %>%
    pull(count) %>%
    matrix(nrow = 2, ncol = 2, byrow = TRUE, dimnames = list(labexp, labout))
age10 <- filter(sorteddat, level == "age 10 years") %>%
    pull(count) %>%
    matrix(nrow = 2, ncol = 2, byrow = TRUE, dimnames = list(labexp, labout))
age11_12 <- filter(sorteddat, level == "age 11-12 years") %>%
    pull(count) %>%
    matrix(nrow = 2, ncol = 2, byrow = TRUE, dimnames = list(labexp, labout))
datarr <- array(
    c(age8, age9, age10, age11_12),
    dim = c(2, 2, 4),
    dimnames = list(labexp, labout, lablev)
)
print(datarr)

, , age 8 years

                caries no caries
not fluoridated      8        23
fluoridated          5        25

, , age 9 years

                caries no caries
not fluoridated     17        33
fluoridated          0        17

, , age 10 years

                caries no caries
not fluoridated     24        14
fluoridated          5        13

, , age 11-12 years

                caries no caries
not fluoridated     29        25
fluoridated          5        16



## Stratum-specific odds ratios

Calculate the stratum-specific odds ratio.

Note, we do not calculate the stratum-specific odds ratio for level age 9 years, given the presence of the 0 count.

In [6]:
# age 8 years
oddsratio(datarr[, , 1])

,oddsratio,stderr,lcb,ucb
exposure 1 (-),NA,NA,NA,NA
exposure 2 (+),1.73913,0.639123,0.4969465,6.086318


In [7]:
# age 10 years
oddsratio(datarr[, , 3])

,oddsratio,stderr,lcb,ucb
exposure 1 (-),NA,NA,NA,NA
exposure 2 (+),4.457143,0.6245145,1.310596,15.15808


In [8]:
# age 11-12 years
oddsratio(datarr[, , 4])

,oddsratio,stderr,lcb,ucb
exposure 1 (-),NA,NA,NA,NA
exposure 2 (+),3.712,0.5805022,1.189826,11.58063


## Crude odds ratio

In [9]:
oddsratio_crude(datarr)

,oddsratio,stderr,lcb,ucb
exposure 1 (-),NA,NA,NA,NA
exposure 2 (+),3.886316,0.3226422,2.064926,7.314281


## Tarone's test for homogeneity

In [10]:
BreslowDayTest(datarr, correct = TRUE)


	Breslow-Day Test on Homogeneity of Odds Ratios (with Tarone
	correction)

data:  datarr
X-squared = 3.9603, df = 3, p-value = 0.2658


## Mantel-Haenszel odds ratio and chi-squared test

In [11]:
mantelhaen.test(datarr)


	Mantel-Haenszel chi-squared test with continuity correction

data:  datarr
Mantel-Haenszel X-squared = 16.528, df = 1, p-value = 4.794e-05
alternative hypothesis: true common odds ratio is not equal to 1
95 percent confidence interval:
 2.067622 7.853659
sample estimates:
common odds ratio 
         4.029689 
